# Feature Reduction - Eliminating unwanted columns manually

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
import os
from io import StringIO

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from azure.storage.blob import BlobServiceClient
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:
%aimport src.custom_transformers
from src.custom_transformers import (
    DFNanThresholdColumnDropper,
    DFColumnDropper,
    DFColumnFilterList,
    DFColumnMapper,
    DFNonUniqueValColDropper,
)

In [ ]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

## About

We will start by removing columns from the data that can't be used in analysis for a variety of reasons (summarized here). Details about the columns are available [here](https://www.rubydoc.info/gems/lending_club/0.0.2/LendingClub/Loan).

## User Inputs

Input values are defined below

In [ ]:
raw_data_path = "data/raw/lending_club_loans.csv"
cloud_storage = "no"

## Load data

In [ ]:
if cloud_storage == "yes":
    az_storage_container_name = "myconedesx7"
    conn_str = (
        "DefaultEndpointsProtocol=https;"
        f"AccountName={os.getenv('AZURE_STORAGE_ACCOUNT')};"
        f"AccountKey={os.getenv('AZURE_STORAGE_KEY')};"
        f"EndpointSuffix={os.getenv('ENDPOINT_SUFFIX')}"
    )
    blob_service_client = BlobServiceClient.from_connection_string(conn_str=conn_str)

    blobstrings = {}
    for blob_name in ["blobedesz38"]:
        blob_client = blob_service_client.get_blob_client(
            container=az_storage_container_name, blob=blob_name
        )
        blobstring = blob_client.download_blob().content_as_text()
    loans_2007 = pd.read_csv(StringIO(blobstring), skiprows=1, low_memory=False)
else:
    loans_2007 = pd.read_csv(raw_data_path, skiprows=1, low_memory=False)

In [ ]:
# Set aside 33% as test data
loans_2007, _ = train_test_split(loans_2007, test_size=0.33, random_state=4321)
loans_2007 = loans_2007.reset_index(drop=True)

## Manual feature reduction

Here, we'll manually eliminate certain columns that are either
- unusable for modeling purposes, such as ZIP code
- populated after a single loan record is created
  - this is fine for record-keeping purposes, but since they cannot be known until some time after the loan begins to be funded/paid off they cannot be used to *predict* whether the corresponding loan will be paid off on time or not

### Columns missing more than half of their values

In [ ]:
nan_threshold = 0.5

### Columns not used here

In [ ]:
non_useful_cols = ["url", "desc"]

### Columns with a date

In [ ]:
datetime_cols = ["issue_d", "last_pymnt_d"]

### First group of columns

In [ ]:
# class DFNanThresholdColumnDropper(TransformerMixin):
#     def __init__(self, threshold):
#         self.threshold = threshold

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         # assumes X is a DataFrame
#         return X.dropna(thresh=self.threshold * len(X), axis=1)

#     def fit_transform(self, X, y=None, **kwargs):
#         self = self.fit(X, y)
#         return self.transform(X)

In [ ]:
# class DFColumnDropper(TransformerMixin):
#     def __init__(self, columns):
#         self.columns = columns

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         # assumes X is a DataFrame
#         cols_to_drop = []
#         for c in list(X):
#             for cd in self.columns:
#                 if cd in c:
#                     cols_to_drop.append(cd)
#         if cols_to_drop:
#             return X.drop(cols_to_drop, axis=1)
#         else:
#             return X

#     def fit_transform(self, X, y=None, **kwargs):
#         self = self.fit(X, y)
#         return self.transform(X)

In [ ]:
cols_one_eighteen = [
    "id",
    "member_id",
    "funded_amnt",
    "funded_amnt_inv",
    "grade",
    "sub_grade",
    "emp_title",
]

The `*id` columns are randomly generated so won't help during further analysis. The `*grade` columns overlap with `int_rate`. `emp_title` is a very [high cardinality column](https://en.wikipedia.org/wiki/Cardinality_(SQL_statements)) and requires considerable processing to get something meaningful values/groups. Other columns suffer from [lookahead bias](https://corporatefinanceinstitute.com/resources/knowledge/finance/look-ahead-bias/).

### Second group of columns

In [ ]:
cols_eighteen_thirtysix = [
    "zip_code",
    "out_prncp",
    "out_prncp_inv",
    "total_pymnt",
    "total_pymnt_inv",
    "total_rec_prncp",
]

`zip_code` overlaps with `addr_state` and is partially [desinsitized](https://patents.google.com/patent/CN106203145A/en) for privacy reasons. Other columns in this group suffer from lookahead bias.

### Third group of columns

In [ ]:
cols_thirtyseven_end = [
    "total_rec_int",
    "total_rec_late_fee",
    "recoveries",
    "collection_recovery_fee",
    "last_pymnt_amnt",
]

All columns in this group suffer from lookahead bias.

### Drop columns

Having defined lists of columns to be dropped, we'll use a pipeline to drop them below

In [ ]:
pipe = Pipeline(
    [
        ("nan", DFNanThresholdColumnDropper(nan_threshold)),
        ("nouse", DFColumnDropper(non_useful_cols)),
        ("dtime", DFColumnDropper(datetime_cols)),
        ("c1", DFColumnDropper(cols_one_eighteen)),
        ("c2", DFColumnDropper(cols_eighteen_thirtysix)),
        ("c3", DFColumnDropper(cols_thirtyseven_end)),
    ]
)
loans_2007 = pipe.fit_transform(loans_2007)
loans_2007.head()

## Visualize distribution of labels

We'll now examine the class distribution of loan status

In [ ]:
def customize_splines(ax: plt.axis) -> plt.axis:
    ax.spines["left"].set_edgecolor("black")
    ax.spines["left"].set_linewidth(2)
    ax.spines["bottom"].set_edgecolor("black")
    ax.spines["bottom"].set_linewidth(2)
    ax.spines["top"].set_edgecolor("lightgrey")
    ax.spines["top"].set_linewidth(1)
    ax.spines["right"].set_edgecolor("lightgrey")
    ax.spines["right"].set_linewidth(1)
    return ax

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
loans_2007["loan_status"].value_counts().to_frame().sort_values(
    by=["loan_status"], ascending=True
).plot(ax=ax, kind="barh", zorder=3)
ax.get_legend().remove()
ax.grid(which="both", axis="both", color="lightgrey", zorder=0)
ax.set_title("Loan Status", loc="left", fontweight="bold")
_ = customize_splines(ax)

Since we're only looking for binary classification, predicting if a loan is either paid off or not, we'll filter out other entries in the `loan_status` (i.e. the labels) column. By doing this, we will not consider loans that are currently active.

In [ ]:
# class DFColumnFilterList(TransformerMixin):
#     def __init__(self, column_name, column_values):
#         self.column_name = column_name
#         self.column_values = column_values

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         # assumes X is a DataFrame
#         # return X[pd.DataFrame(X[self.column_name].tolist()).isin(self.column_values).any(1)]
#         return X.loc[X[self.column_name].isin(self.column_values)]

#     def fit_transform(self, X, y=None, **kwargs):
#         self = self.fit(X, y)
#         return self.transform(X)

In [ ]:
loan_status = ["Fully Paid", "Charged Off"]

In [ ]:
pipe = Pipeline(
    [
        ("filter", DFColumnFilterList("loan_status", loan_status)),
    ]
)
loans_20072 = pipe.fit_transform(loans_2007)
loans_20072.head()

In [ ]:
loans_20071 = loans_2007[
    (loans_2007["loan_status"] == "Fully Paid")
    | (loans_2007["loan_status"] == "Charged Off")
]

In [ ]:
assert loans_20071.equals(loans_20072)

Below, we'll show the class balance with red being `Charged Off` (not paid on time) and green being `Fully Paid` (paid on time)

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))
sns.countplot(x="loan_status", data=loans_20071, ax=axs[0], palette=["red", "green"])
axs[0].set_title("Frequency of each Loan Status")
axs[0].set_ylabel(None)
axs[0].set_xlabel(None)
loans_20071.loan_status.value_counts().plot(
    x=None, y=None, kind="pie", ax=axs[1], autopct="%1.2f%%", colors=["green", "red"]
)
axs[1].set_title("Percentage of each Loan status")
axs[1].set_ylabel(None)

Eventually, we'll need to convert these labels to numeric values so we'll make this replacement here with `0` being mapped to `Charged Off` and `1` to `Fully Paid`

In [ ]:
# class DFColumnMapper(TransformerMixin):
#     def __init__(self, mapping_dict):
#         self.mapping_dict = mapping_dict

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         # assumes X is a DataFrame
#         return X.replace(self.mapping_dict)

#     def fit_transform(self, X, y=None, **kwargs):
#         self = self.fit(X, y)
#         return self.transform(X)

In [ ]:
mapping_dictionary = {"loan_status": {"Fully Paid": 1, "Charged Off": 0}}
loans_200711 = loans_20071.replace(mapping_dictionary)

In [ ]:
pipe = Pipeline(
    [
        ("colmap", DFColumnMapper(mapping_dictionary)),
    ]
)
loans_200722 = pipe.fit_transform(loans_20071)
assert loans_200722.equals(loans_200711)

## Drop columns with infrequently occurring values

Next, we'll remove categorical columns from the data with a single unique value, since these won't be useful to a model.

In [ ]:
# loans_200711 = loans_200711.loc[:, loans_200711.apply(pd.Series.nunique) != 1]
single_value_columns = []
for col in loans_200711:
    non_null = loans_200711[col].dropna()
    unique_non_null = non_null.unique()
    num_true_unique = len(unique_non_null)
    if num_true_unique == 1:
        single_value_columns.append(col)
loans_2007111 = loans_200711.drop(single_value_columns, axis=1)
single_value_columns

Note that the class method `.unique()` was used to count the number of unique values in a column but, missing values in that column, had to be dropped first since `.unique()` also counts missing values as unique values.

Next, we'll drop columns where there are more than one, but fewer than five, unique values. We've assumed here that an infrequency threshold of five is good enough, but this threshold can be tweaked later if deemed necessary

In [ ]:
for col in loans_2007111.columns:
    if len(loans_2007111[col].unique()) < 4:
        display(loans_2007111[col].value_counts().to_frame())

The `pymnt_plan` column has a single occurrence of `'y'` so it can be dropped

In [ ]:
four_or_less_value_columns = ["pymnt_plan"]

In [ ]:
loans_20071111 = loans_2007111.drop(
    columns=four_or_less_value_columns, axis=1, errors="ignore"
)

In [ ]:
# class DFNonUniqueValColDropper(TransformerMixin):
#     def __init__(self, num_non_unique_vals):
#         self.num_non_unique_vals = num_non_unique_vals

#     def fit(self, X, y=None):
#         return self

#     def transform(self, X):
#         # assumes X is a DataFrame
#         X = X.loc[:, X.apply(pd.Series.nunique) > self.num_non_unique_vals]
#         return X

#     def fit_transform(self, X, y=None, **kwargs):
#         self = self.fit(X, y)
#         return self.transform(X)

In [ ]:
pipe = Pipeline(
    [
        ("onevals", DFNonUniqueValColDropper(1)),
        ("fourvals", DFColumnDropper(four_or_less_value_columns)),
    ]
)
loans_2007222 = pipe.fit_transform(loans_200722)
assert loans_20071111.equals(loans_2007222)
display(loans_2007222.shape)

## Verify pipeline-based and manual methods of feature reduction agree

Here, we'll re-load the raw data and verify that the manual and pipeline-based approaches to dropping, filtering and replacing values in columns agree with eachother

In [ ]:
if cloud_storage == "yes":
    az_storage_container_name = "myconedesx7"
    conn_str = (
        "DefaultEndpointsProtocol=https;"
        f"AccountName={os.getenv('AZURE_STORAGE_ACCOUNT')};"
        f"AccountKey={os.getenv('AZURE_STORAGE_KEY')};"
        f"EndpointSuffix={os.getenv('ENDPOINT_SUFFIX')}"
    )
    blob_service_client = BlobServiceClient.from_connection_string(conn_str=conn_str)

    blobstrings = {}
    for blob_name in ["blobedesz38"]:
        blob_client = blob_service_client.get_blob_client(
            container=az_storage_container_name, blob=blob_name
        )
        blobstring = blob_client.download_blob().content_as_text()
    loans_2007 = pd.read_csv(StringIO(blobstring), skiprows=1, low_memory=False)
else:
    loans_2007 = pd.read_csv(raw_data_path, skiprows=1, low_memory=False)

In [ ]:
# Set aside 33% as test data
loans_2007, _ = train_test_split(loans_2007, test_size=0.33, random_state=4321)
loans_2007 = loans_2007.reset_index(drop=True)

In [ ]:
pipe = Pipeline(
    [
        ("nan", DFNanThresholdColumnDropper(nan_threshold)),
        ("nouse", DFColumnDropper(non_useful_cols)),
        ("dtime", DFColumnDropper(datetime_cols)),
        ("c1", DFColumnDropper(cols_one_eighteen)),
        ("c2", DFColumnDropper(cols_eighteen_thirtysix)),
        ("c3", DFColumnDropper(cols_thirtyseven_end)),
        (
            "mapstatus",
            DFColumnFilterList("loan_status", loan_status),
        ),
        ("colmap", DFColumnMapper(mapping_dictionary)),
        ("onevals", DFNonUniqueValColDropper(1)),
        ("fourvals", DFColumnDropper(four_or_less_value_columns)),
    ]
)

In [ ]:
loans_2007_pipe_transformed = pipe.fit_transform(loans_2007)
print(loans_2007_pipe_transformed.shape)
display(loans_2007_pipe_transformed.head())

We now verify that both approaches produce the same filtered dataset, for the next phase of processing

In [ ]:
assert loans_2007_pipe_transformed.equals(loans_2007222)